# Overview

This tutorial shows you how the library interacts with custom mods. To do so we create a generic mod that runs code on the vehicle and game engine side.

In this tutorial we differentiate between mod and lua module:

mod - refers to a collection of files that change the behavior of the game (see the zip file)

lua module - refers to a lua 'library' that has a global name containing a table 


## Setup

These are the usual imports to create a simple scenario. As the point is to show the mod support, nothing is happening in the scenario. The mod only prints some text to the console, so take a look at the console or the log file you can find in the userpath `%localappdata%\BeamNG.tech\<version>`. Note that the console only flushes the last line to the log file if another console output line is generated.



In [1]:
import os
import zipfile
from pathlib import Path

from beamngpy import BeamNGpy, Scenario, Vehicle, set_up_simple_logging
from beamngpy.beamngcommon import ack

In [2]:
set_up_simple_logging()
beamng = BeamNGpy('localhost', 64256)

scenario = Scenario('smallgrid', 
                    "On how to use custom mods")

etk = Vehicle('ego_vehicle', model='etk800', licence='AI', extensions=["vehicleEngineCode"])
scenario.add_vehicle(etk, pos=(0,0,0), rot_quat=(0, 0, 1, 0))

pickup = Vehicle('some_vehicle', model='pickup', licence='AI', extensions=["vehicleEngineCode"])
scenario.add_vehicle(pickup, pos=(5,0,0), rot_quat=(0, 0, 1, 0))

2022-01-24 11:03:55,142 |INFO     |beamngpy                      |Started BeamNGpy logging.


## Creating a Simple Mod

Mods can be installed by creating a zip directory in the ".../Documents/BeamNG.reasearch/mods/" folder. They have to recreate the exact same file structure as you find in the game directory in "BeamNG.tech.vX.X.X.X/lua".

What happens here is that the two lua files in the BeamNGpy directory are zipped into a mod.

In [3]:
userpath = Path(BeamNGpy.read_effective_userpath(beamng.user))

# setting up mod
myModPath = userpath / 'mods' / 'genericResearchMod.zip' 
myModPath.parent.mkdir(parents=True, exist_ok=True)

geCode = 'gameEngineCode.lua'
zipGEpath = str(Path('lua') / 'ge' / 'extensions' / 'util' / geCode)
veCode = 'vehicleEngineCode.lua'
zipVEpath = str(Path('lua') / 'vehicle' / 'extensions' / veCode)
localDir = Path(os.path.abspath('.'))

with zipfile.ZipFile(str(myModPath), 'w') as ziph:
    ziph.write(localDir / geCode, arcname=zipGEpath)
    ziph.write(localDir / veCode, arcname=zipVEpath)

## Testing the mod

To test the mod we start BeamNG.tech and give the python BeamNG class the location of the gameengine mod within the "genericResearchMod.zip/lua/ge/extensions/" directory. This is necessary to register the file as its own lua module within the game.

After registration, it is available as util_gameEngineCode within the game - try typing `dump(util_gameEngineCode)` in the game's command prompt

In [4]:
bng = beamng.open(extensions=["util/gameEngineCode"])
scenario.make(beamng)
bng.load_scenario(scenario)
bng.start_scenario()

2022-01-24 11:03:57,555 |INFO     |beamngpy.BeamNGpy             |Opening BeamNGpy instance.
2022-01-24 11:03:57,558 |INFO     |beamngpy.BeamNGpy             |Started BeamNG.
2022-01-24 11:04:00,620 |INFO     |beamngpy.BeamNGpy             |Terminating BeamNG.tech process.
2022-01-24 11:04:00,699 |INFO     |beamngpy.BeamNGpy             |Started BeamNG.
2022-01-24 11:04:10,700 |INFO     |beamngpy.BeamNGpy             |Connecting to BeamNG.tech at: (localhost, 64256)
2022-01-24 11:04:11,619 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2022-01-24 11:04:11,619 |INFO     |beamngpy.BeamNGpy             |BeamNGpy successfully connected to BeamNG.
2022-01-24 11:04:17,422 |INFO     |beamngpy.BeamNGpy             |Loaded map.
2022-01-24 11:04:17,557 |INFO     |beamngpy.Vehicle              |Vehicle ego_vehicle connected to simulation.
2022-01-24 11:04:17,589 |INFO     |beamngpy.Vehicle              |Attempting to connect to vehicle 'ego_vehicle'
2022-01-24 11

## How to Call Lua Module Functions

BeamNG.tech and BeamNGpy communicate via sockets. 
Function parameters are send with the help of python dictionaries that are available as lua tables in the game. 
The mini mod from this tutorial follows the convention the BeamNGpy library established: Every value for the 'type' key helps identifying the appropriate handler, which for 'Foo' is expected to be 'handleFoo'.
The checkmessage function in researchGE.lua and researchVE.lua checks whether a corresponding function is locally available and, if not, calls the `onSocketMessage` hook for every extension. 

## Using Acknowledgements

In order to use the beamngcommon `ack` function decorator, the first function argument has to always be the object that also handles the socket. That is either the instance of the python BeamNG class or a vehicle object. 

## Data Transfer between BeamNGpy and BeamNG

Any return value needs to be send with the help of the socket. See here an example on how to do it on the game engine side. The code for the vehicle side is the same, it just needs to be implemented in researchVE.lua.



In [5]:
@ack('FooAck')
def callFoo(bng):
    """ Executes handleFoo defined in gameEngineCode.lua.
    """
    data = dict(type='Foo', someName = 'YourName')
    bng.send(data)

@ack('BarAck')
def callBar(veh):
    """ Executes handlebar defined in VehicleEngineCode.lua.
    Here the code is executed in the VM of the etk800.
    """
    data = dict(type='Bar', text = 'lorem ipsum...' )
    veh.send(data)

def dataExchange(bng):
    """Demonstrates how to transfer in game data to the python side.
    """
    data = dict(type='GetListOfVehicleModels')
    bng.send(data)
    response = bng.recv()
    assert response['type'] == 'ListOfVehicleModels'
    print('List of spawned vehicle models: ', response['data'])

In [6]:
callFoo(bng)

In [7]:
callBar(etk)

In [8]:
dataExchange(bng)

List of spawned vehicle models:  ['etk800', 'pickup']


### Don't see anything?

If you have trouble finding the messages in the log file or command prompt, search for 'gameEngineCode' or 'vehicleEngineCode'. These are the log tags of the respective modules.

In [9]:
bng.close()

2022-01-24 11:04:38,162 |INFO     |beamngpy.BeamNGpy             |Closing BeamNGpy instance.
2022-01-24 11:04:38,163 |INFO     |beamngpy.Vehicle              |Disconnected from vehicle ego_vehicle and detached sensors.
2022-01-24 11:04:38,164 |INFO     |beamngpy.Vehicle              |Disconnected from vehicle some_vehicle and detached sensors.
2022-01-24 11:04:38,165 |INFO     |beamngpy.BeamNGpy             |Terminating BeamNG.tech process.
